In [1]:
import numpy as np
from osgeo import gdal
import torch
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib import patches
from osgeo import gdal, osr

In [2]:
SLICE_SIZE = 512
MM_TO_PIXEL = 4.11  # Check from report needs to be set everytime
PATH_TO_MODEL = '../weights/best.pt'
PATH_TO_ORTHOPHOTO = '../scripts/B2K_5_ortho.tiff'
PATH_TO_DEM = '../scripts/Bhagina to Kajra 5 DEM.tif'


In [3]:
# convert the final bbox coords wrt to the ones on the DEM
def coord_converter(xmin_photo, ymin_photo, xmax_photo, ymax_photo, dataset_photo, dataset_dem):

    # Get geotransform of the orthophoto
    geotransform_photo = dataset_photo.GetGeoTransform()

    # Get the real world coordinates
    x_min_real = geotransform_photo[0] + xmin_photo * \
        geotransform_photo[1] + ymin_photo * geotransform_photo[2]
    y_min_real = geotransform_photo[3] + xmin_photo * \
        geotransform_photo[4] + ymin_photo * geotransform_photo[5]

    x_max_real = geotransform_photo[0] + xmax_photo * \
        geotransform_photo[1] + ymax_photo * geotransform_photo[2]
    y_max_real = geotransform_photo[3] + xmax_photo * \
        geotransform_photo[4] + ymax_photo * geotransform_photo[5]

    # use real word coords on the geotagged orthophoto
    geotransform_photo_dem = dataset_dem.GetGeoTransform()

    srs = osr.SpatialReference()
    srs.ImportFromWkt(dataset_dem.GetProjection())

    coord_transform = osr.CoordinateTransformation(srs, srs.CloneGeogCS())

    # Convert the real-world coordinates to projected coordinates
    x_proj_min, y_proj_min, _ = coord_transform.TransformPoint(
        x_min_real, y_min_real)
    x_proj_max, y_proj_max, _ = coord_transform.TransformPoint(
        x_max_real, y_max_real)

    td0 = geotransform_photo_dem[0]
    td1 = geotransform_photo_dem[1]
    td2 = geotransform_photo_dem[2]
    td3 = geotransform_photo_dem[3]
    td4 = geotransform_photo_dem[4]
    td5 = geotransform_photo_dem[5]

    x_min_dem = int(
        ((x_proj_min - td0)*td5 - (y_proj_min - td3)*td2)/(td1*td5 - td2*td4)
    )

    y_min_dem = int(
        ((x_proj_min - td0)*td4 - (y_proj_min - td3)*td1)/(td2*td4 - td1*td5)
    )

    x_max_dem = int(
        ((x_proj_max - td0)*td5 - (y_proj_max - td3)*td2)/(td1*td5 - td2*td4)
    )

    y_max_dem = int(
        ((x_proj_max - td0)*td4 - (y_proj_max - td3)*td1)/(td2*td4 - td1*td5)
    )

    # Note the last two items are the real world coordinates of the centroid of the bbox to identify and locate each pothole
    return x_min_dem, y_min_dem, x_max_dem, y_max_dem, (x_min_real + x_max_real)/2, (y_min_real + y_max_real)/2


In [4]:
dataset_ortho = gdal.Open(PATH_TO_ORTHOPHOTO)
dataset_dem = gdal.Open(PATH_TO_DEM)

In [5]:
dem_array = np.array(dataset_dem.ReadAsArray())

In [6]:
def calculate_slice_bboxes(
    orthophoto_dataset,
    image_height: int,
    image_width: int,
    slice_height: int = SLICE_SIZE,
    slice_width: int = SLICE_SIZE,
    overlap_height_ratio: float = 0.2,
    overlap_width_ratio: float = 0.2):

    def check_garbage_slice(dataset, xmin, ymin, xmax, ymax):

        image = np.array(dataset.ReadAsArray(xmin, ymin, xmax - xmin, ymax - ymin))
        image = image[0:3, :, :]
        corner1 = image[:, 0, 0]
        corner2 = image[:, ymax - ymin - 1, xmax - xmin - 1]
        corner3 = image[:, 0, xmax - xmin - 1]
        corner4 = image[:, ymax - ymin - 1, 0]
        reject = np.array([255, 255, 255])
        corners = [corner1, corner2, corner3, corner4]
        for corner in corners:
            if (corner == reject).all():
                return True
        return False
    
    slice_bboxes = []
    y_max = y_min = 0
    y_overlap = int(overlap_height_ratio * slice_height)
    x_overlap = int(overlap_width_ratio * slice_width)
    while y_max < image_height:
        x_min = x_max = 0
        y_max = y_min + slice_height
        while x_max < image_width:
            x_max = x_min + slice_width
            if y_max > image_height or x_max > image_width:
                xmax = min(image_width, x_max)
                ymax = min(image_height, y_max)
                xmin = max(0, xmax - slice_width)
                ymin = max(0, ymax - slice_height)
                if not check_garbage_slice(orthophoto_dataset, xmin, ymin, xmax, ymax):
                    slice_bboxes.append([xmin, ymin, xmax, ymax])

            else:
                if not check_garbage_slice(orthophoto_dataset, x_min, y_min, x_max, y_max):
                    slice_bboxes.append([x_min, y_min, x_max, y_max])

            x_min = x_max - x_overlap
        y_min = y_max - y_overlap

    return slice_bboxes


In [7]:
slices = calculate_slice_bboxes(
    orthophoto_dataset=dataset_ortho,
    image_height=dataset_ortho.RasterYSize,
    image_width=dataset_ortho.RasterXSize
    )

In [8]:
len(slices)

300

In [12]:
def get_check_bbox_coords(bboxes, result, image_dim, slice):

    xmin_slice, ymin_slice, xmax_slice, ymax_slice = slice
    # print(image_dim)

    boxes = result.xyxyn[0]
    for box in boxes:
        box = np.array(box)
        xmin, ymin, xmax, ymax, confidence, label = box

        if label == 4 and confidence > 0.5:          # index label of the pothole in the in the model
            # result.show()
            # print('before ', xmin, ymin, xmax, ymax)
            xmin *= image_dim
            xmax *= image_dim
            ymin *= image_dim
            ymax *= image_dim

            # print('after ', xmin, ymin, xmax, ymax)

            xmin, ymin, xmax, ymax = int(xmin), int(ymin), int(xmax), int(ymax)
            # print('after after ', xmin, ymin, xmax, ymax)
            # Get coordinates wrt original orthophoto
            xmin += xmin_slice
            ymin += ymin_slice
            xmax += xmin_slice
            ymax += ymin_slice

            bbox = (xmin, ymin, xmax, ymax)
            bboxes.append(bbox)

        else:
            continue

In [13]:
def detector(slices):

    bboxes = []
    print("Running the detector model...")
    model = torch.hub.load('ultralytics/yolov5', 'custom', path=PATH_TO_MODEL, source='github')
    for slice in tqdm(slices):
        # print('slice ', slice)
        xmin, ymin, xmax, ymax = slice
        sliced_image = np.array(dataset_ortho.ReadAsArray(xmin, ymin, xmax - xmin, ymax - ymin))
        sliced_image = sliced_image[0:3, :, :]
        # print(sliced_image.shape)
        result = model(sliced_image)
        get_check_bbox_coords(bboxes, result, sliced_image.shape[1], slice)   # adds the bbox coordinates to the list wrt orthophoto
    return  bboxes

In [14]:
bbox_vals = detector(slices=slices)

Running the detector model...


Using cache found in /Users/adityadandwate/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-8-25 Python-3.11.0 torch-2.0.0 CPU

Fusing layers... 
Model summary: 157 layers, 7029004 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
100%|██████████| 300/300 [16:19<00:00,  3.27s/it] 


In [78]:
len(bbox_vals)

41

In [81]:
def convert_to_dem_coords(bboxes):
    
    final_bboxes_dem = []
    for box in bboxes:
        # print(box)
        final_box_dem = coord_converter(box[0], box[1], box[2], box[3], dataset_ortho, dataset_dem)
        final_bboxes_dem.append(final_box_dem)

    return final_bboxes_dem

In [82]:
final_bboxes_dem = convert_to_dem_coords(bbox_vals)

In [45]:
len(final_bboxes_dem)

41

In [46]:
def show_image_3(image):

    plt.imshow(image)
    plt.savefig("test2.png")


In [83]:
def calculate_volume_and_maxdepth(final_bboxes_dem):
    """
    Append the volume and maximum depth of the pothole to the list
    """
    volume_max_depth = []
    area = MM_TO_PIXEL*MM_TO_PIXEL

    for box in final_bboxes_dem:

        x_min_dem, y_min_dem, x_max_dem, y_max_dem, x_real, y_real = box
        print(x_min_dem, y_min_dem, x_max_dem, y_max_dem)
        sliced_dem = dem_array[y_min_dem:y_max_dem,
                                    x_min_dem:x_max_dem]

        max_depth = np.max(sliced_dem)
        min_depth = np.min(sliced_dem)

        max_depth_floor = (max_depth - min_depth)
        
        # We shall use our min depth as the floor depth
        volume = 0
        for i in range(sliced_dem.shape[0]):
            for j in range(sliced_dem.shape[1]):

                depth_wrt_floor = sliced_dem[i][j] - min_depth
                volume += area*depth_wrt_floor

        # contains the volume, the max depth, and the real world coords of the potholes
        volume_max_depth.append(
            (volume, max_depth_floor, (x_real, y_real)))

    return volume_max_depth


In [84]:
volume_max_depth = calculate_volume_and_maxdepth(final_bboxes_dem)

1421 974 1452 995
2101 1580 2126 1601
2205 1443 2226 1468
2888 2263 2906 2278
6631 5265 6657 5282
7604 6059 7617 6072
10366 8154 10386 8175
10798 8435 10817 8454
11583 9448 11614 9474
11844 9461 11863 9478
12150 9943 12179 9967
13154 10822 13183 10845
16722 13997 16745 14019
16983 14348 17007 14368
19457 16445 19478 16466
21198 18347 21226 18369
23913 21010 23965 21045
24191 21480 24209 21498
24447 21468 24464 21487
24880 21947 24898 21965
25272 22356 25299 22378
26204 23142 26215 23152
28559 25779 28574 25793
30320 27449 30352 27476
31611 28985 31634 29009
31726 28941 31757 28973
31713 29042 31729 29055
32942 30136 33057 30280
34463 31407 34488 31430
34289 31714 34345 31758
37368 34519 37392 34538
37740 34681 37772 34712
37651 34829 37672 34851
38369 35158 38387 35183
39229 36297 39296 36337
39787 36491 39944 36551
43673 40008 43719 40050
44921 40912 44935 40926
46945 42502 46994 42548
46800 42787 46828 42825
48977 44281 49008 44308


In [85]:
len(volume_max_depth)

41

In [88]:
def calculate_severity(volume_max_depth):

    final_results = []

    # using final bboxes ortho and not final bboxes dem for consistency across the mm to pixel ratio
    for i, box in enumerate(bbox_vals):

        xmin, ymin, xmax, ymax = box
        xlength = xmax - xmin
        ylength = ymax - ymin

        xlength *= MM_TO_PIXEL
        ylength *= MM_TO_PIXEL

        width = max(xlength, ylength)

        volume, depth, real_coords = volume_max_depth[i]

        # depth value is in metres so convert to mm
        depth *= 1000

        # Severity is either small or medium
        if width >= 500:
            if depth >= 25 and depth <= 50:
                final_results.append((volume, 1, real_coords))

            elif depth > 50:
                final_results.append((volume, 2, real_coords))
            else:
                    # Classify as medium in undefined edge case
                final_results.append((volume, 1, real_coords))

        else:
            # Classify as small in all undefined edge cases in this category
            if depth <= 25:
                final_results.append((volume, 0, real_coords))
            else:
                final_results.append((volume, 1, real_coords))

    return final_results


In [90]:
final_results = calculate_severity(volume_max_depth)


[(80.55863522644033, 0, (75.68391125164004, 28.3558642618316)),
 (51.81814574890142, 0, (75.68395613443825, 28.355828706966598)),
 (44.86551657714844, 0, (75.68396288011729, 28.3558366243386)),
 (22.31728494873047, 0, (75.6840080943985, 28.3557888415324)),
 (28.772966354370126, 0, (75.68425649118454, 28.355612723990802)),
 (21.051717764282234, 0, (75.68432058342259, 28.3555662607466)),
 (21.39659126586915, 0, (75.68450391009574, 28.3554431602738)),
 (19.610363040161147, 0, (75.68453251707865, 28.355426724396)),
 (65.70639239501949, 0, (75.6845849411151, 28.355367124179)),
 (20.458370379638684, 0, (75.68460184674808, 28.3553666256778)),
 (68.5189930297852, 0, (75.68462244841655, 28.3553381524622)),
 (110.39560592651365, 0, (75.6846890267771, 28.355286616235198)),
 (53.60849802246094, 0, (75.68492535758193, 28.3551004846842)),
 (34.640455435180684, 0, (75.68494269414285, 28.355079972825997)),
 (23.220451428222667, 0, (75.68510659591219, 28.3549569310004)),
 (43.05505957031249, 0, (75.685